# Atcoder Heuristic
競技プログラミングのコンテストサイトAtcoderにおけるヒューリスティック系の問題に対する基本的な解法をまとめる。

Referece
- 競技プログラミングの鉄則
    - 本

## 局所探索と焼きなまし
問題に対して解を生成した後に、その解を少しずつ変化させていくことでより良い解を探索する手法。
使える条件としてざっくりと以下のような
- 解が与えられた情報から生成できる
- 解に対して評価ができる
    - 評価の方法は直接的でも、自力で作成しても良い
ときに使える

以下の例題を考える  
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_at  
いわゆる巡回セールスマン問題と呼ばれる問題

In [ ]:
import random
import numpy as np
import time

In [ ]:
# 入力（本来は標準入力が与えられるが、ここではサイトから入力例をコピーして使う）
N = 7
xys = [[1,1],[4,1],[2,5],[3,4],[3,2],[4,2],[5,5]]

In [ ]:
# util functions 
def get_distance(idx_p,idx_q,xy):
    return ((xy[idx_p][0] - xy[idx_q][0])**2 + (xy[idx_p][1] - xy[idx_q][1])**2)**0.5

def get_total_distance(route,xy):
    total_distance = 0
    current_place = 1
    for j in route:
        total_distance += get_distance(current_place-1,j-1,xy)
        current_place = j
    return total_distance

In [ ]:
# 局所探索
# 初期解の生成
ans = [1] + list(range(2,N+1)) + [1]

# 今回の問題では解の途中のランダムな2点を選んでその間の都市の巡回順序を逆転させる
# 他の問題に対してここをどう考えるかがポイント
def random_reverse(route):
    idx_p = random.randint(1,N-1)
    idx_q = random.randint(idx_p+1,N)
    route[idx_p:idx_q] = reversed(route[idx_p:idx_q])
    return route

current_score = get_total_distance(ans,xys)
start_time = time.time()
while time.time() - start_time < 1.0:
    new_ans = random_reverse(ans.copy())
    new_score = get_total_distance(new_ans,xys)
    if new_score < current_score:
        ans = new_ans
        current_score = new_score

print(current_score)
print(ans)

In [ ]:
# 焼きなまし
# スコアが上がっていなくても確率的に受け入れる
from math import exp 

# 初期解の生成
ans = [1] + list(range(2,N+1)) + [1]

# 今回の問題では解の途中のランダムな2点を選んでその間の都市の巡回順序を逆転させる
# 他の問題に対してここをどう考えるかがポイント
def random_reverse(route):
    idx_p = random.randint(1,N-1)
    idx_q = random.randint(idx_p+1,N)
    route[idx_p:idx_q] = reversed(route[idx_p:idx_q])
    return route

current_score = get_total_distance(ans,xys)
start_time = time.time()
time_step = 0
while time.time() - start_time < 1.0:
    new_ans = random_reverse(ans.copy())
    new_score = get_total_distance(new_ans,xys)

    if new_score < current_score:
        T = 30. - 28.*time_step/200000.
        prob = exp(min(0.,(current_score - new_score)/T))
        if prob > random.random():
            ans = new_ans
            current_score = new_score

    time_step += 1

print(current_score)
print(ans)


## 貪欲法とビームサーチ
問題に対して1手先のスコアが最大となる操作を取り続けるのが貪欲法  
対してビームサーチでは1手先の最善手だけでなく、次善手や次々善手も考慮して最終的に一番スコアが高くなるような操作を取る手法  

以下の例題を考える  
https://atcoder.jp/contests/tessoku-book/tasks/tessoku_book_aw  


In [ ]:
# 入力（本来は標準入力が与えられるが、ここではサイトから入力生成方法に従ってランダムに生成する）
T = 100
pqr = list()
for i in range(100):
    pqr.append(list(np.sort(np.random.permutation(20)[:3] + 1)))

In [ ]:
# 貪欲法
ans = []
x = np.zeros(20)
score = 0

for p,q,r in pqr:
    # Aの操作、Bの操作を比べてスコアの高くなる方を採択
    tmp_a,tmp_b = x.copy(),x.copy()
    # A
    tmp_a[[p-1,q-1,r-1]] += 1
    score_a = 20 - np.count_nonzero(tmp_a)
    # B
    tmp_b[[p-1,q-1,r-1]] -= 1
    score_b = 20 - np.count_nonzero(tmp_b)
    
    if score_a > score_b:
        x[[p-1,q-1,r-1]] += 1
        score += score_a
        ans.append("A")
    else:
        x[[p-1,q-1,r-1]] -= 1
        score += score_b
        ans.append("B")
    
# 回答の出力
# for a in ans:
#     print(a)
# スコア
print(score)

ビームサーチ  
実装においてi手目終了時点での第j位の状態Beam[i][j]には
- 現時点でのスコア
- 現時点での状態（今回なら配列Xの値）
- 現時点で選んだ行動（i手目は操作A,Bのどちらであったか）
- i-1手目時点では第何位の行動を選んだか  

の情報が必要

今回はdictでscore,X,lastmove,lastposで情報を格納する（dataclassとか使ってもいい）

In [ ]:
ans = []
x = np.zeros(20)
score = 0

K = 10 # ビーム幅
beam = [[dict() for _ in range(K)] for _ in range(T+1)]

# 初期状態
beam[0][0]["X"] = x
beam[0][0]["score"] = 0
beam[0][0]["lastmove"] = None
beam[0][0]["lastpos"] = -1
numstate = [0]*(T+1)
numstate[0] = 1

# ビームサーチ
for i in range(1,101):
    p,q,r = pqr[i-1]
    candidate = []
    for n in range(numstate[i-1]):
        # beam[i-1][n]に対して次の操作を行ってスコアや状態などを記録
        tmp_a,tmp_b = beam[i-1][n]["X"].copy(),beam[i-1][n]["X"].copy()
        
        # A
        tmp_a[[p-1,q-1,r-1]] += 1
        score_a = 20 - np.count_nonzero(tmp_a)
        state_dict = {}
        state_dict["X"] = tmp_a
        state_dict["score"] = beam[i-1][n]["score"] + score_a
        state_dict["lastmove"] = "A"
        state_dict["lastpos"] = n
        candidate.append(state_dict)

        # B
        tmp_b[[p-1,q-1,r-1]] -= 1
        score_b = 20 - np.count_nonzero(tmp_b)
        state_dict = {}
        state_dict["X"] = tmp_b
        state_dict["score"] = beam[i-1][n]["score"] + score_b
        state_dict["lastmove"] = "B"
        state_dict["lastpos"] = n
        candidate.append(state_dict)
    
    # scoreでソート
    candidate = sorted(candidate, key=lambda x: x['score'], reverse=True)
    for p in range(min(K,len(candidate))):
        beam[i][p] = candidate[p]
    numstate[i] = min(K,len(candidate))
    
# beam[99][0]から答えをたどる
ans = []
lastpos = 0
for i in range(100,0,-1):
    ans.append(beam[i][lastpos]["lastmove"])
    lastpos = beam[i][lastpos]["lastpos"]
# 回答の出力
# for a in ans[::-1]:
#     print(a)
# score
print(beam[100][0]["score"])